In [1]:
import ipywidgets as widget
from IPython.display import display
from queue import Queue
from threading import Thread


messages = Queue()
recording = Queue()
# Threads are concepts (function) in python that runs in the background without distrubing the main function.
# Queue will let us pass messeges between threads

record_button = widget.Button(
    description = "Record",
    disabled= False,
    Button_style = 'sucess',
    icon = "microphone")

Stop_button = widget.Button(
    description = "Stop",
    disabled = False,
    button_style = 'Warning',
    icon = 'Stop_button'
)
output = widget.Output()



#in this function when the button is pressed we put data on the queue.
def start_recording(data):
    messages.put(True) # Message theat tells the threads what to do 

    with output:
        display("start recording ..")
        record = Thread(target=record_microphone)
        record.start()
        speech = Thread(target=speech_recognitions,args=(output,))
        speech.start()
        
        
 # This function when we press the button takes the message off the queue and the recoring will stop        
def stop_recording(data):
    with output:
        messages.get()
        display("Stopped")
    
record_button.on_click(start_recording)
Stop_button.on_click(stop_recording)

display(record_button, Stop_button, output)



Button(description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='Stop_button', style=ButtonStyle())

Output()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.tr

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions

In [2]:
import pyaudio

CHANNELS = 1 
FRAME_RATE = 16000
RECORD_SECONDS= 2 # every 20 second audio is going to be translated to text
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

# Chunk defines how often we are gonna read from the microphone. 
# input = true means that we are recording from a microphone
# frames per buffer means how often we are going to read data from our microphone



def record_microphone(chunks=1024): # 1024 audio frames at the time
    p = pyaudio.PyAudio()
    stream = p.open(format=AUDIO_FORMAT, channels=CHANNELS, rate=FRAME_RATE, input=True, input_device_index = 0, frames_per_buffer=chunks)
    
    # This part handels the threading work, it says that as long as the stop button is not pressed, record and store the data on thread recording. then
    # after the defined limit empty the frame list 
    # Bascilly every 20 sec we pass our recorded audio to our trancription engine and then start recording another 20 sec
    
    
    # In the first iteration the frames list contains a single data items which is a chunk with 1024 frames it requires approxiametly 312 iteraions
    # until the conditions is reached.
    frames = []
    while not messages.empty():
        
        data = stream.read(chunks)
        frames.append(data)
        
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunks: # if we recorded more than 20 sec of audio
            recording.put(frames.copy())
            frames = []
             

    # these three lines are very important and if missed could cause some problems to the audio parts of your machine
    stream.stop_stream()
    stream.close()
    p.terminate ()           

In [3]:

import subprocess
import json
from vosk import KaldiRecognizer, Model
import time

model = Model(model_name="vosk-model-small-sv-rhasspy-0.15") 
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

output_list = []

def speech_recognitions(output):
    
    while not messages.empty():
        frames = recording.get()  # Fix the queue name to 'recordings'
        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)["text"]  # Use json.loads instead of json.load
        
        cased = subprocess.check_output(' python3 /Users/sina/Python/Github/Python_projects/speech_recognition/vosk-recasepunc-en-0.22/recasepunc.py predict /Users/sina/Python/Github/Python_projects/speech_recognition/vosk-recasepunc-en-0.22/checkpoint', shell=True, text=True, input=text)
        output.append_stdout(cased)
        time.sleep(1)

vosk-model-small-sv-rhasspy-0.15.zip: 100%|██████████| 289M/289M [00:34<00:00, 8.69MB/s]   
LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10:11:12:13:14:15
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 1 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 2 orphan components.
LOG (VoskAPI:Collapse():nnet-utils.cc:1488) Added 1 components, removed 2
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /Users/sina/.cache/vosk/vosk-model-small-sv-rhasspy-0.15/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:279) Loading HCLG from /Users/sina/.cache/vosk/vosk-model-small-sv-rhasspy-0.15/graph/HCLG.fst
LOG (VoskAPI:ReadDataFi